Etapas do Treinamento com Imagens Sintéticas

1) Geração de Dados Sintéticos

As imagens sintéticas serão criadas utilizando o TensorFlow/Keras.

Geramos arrays de imagens aleatórias com ruídos e padrões simulados.

Os rótulos serão atribuídos de forma balanceada.

In [ ]:
!pip install keras-tuner

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models

# Gerando imagens sintéticas (1000 amostras, 224x224, 3 canais RGB)
def generate_synthetic_data(num_samples=1000, img_size=(224, 224, 3), num_classes=2):
    X = np.random.rand(num_samples, *img_size).astype(np.float32)  # Imagens aleatórias
    y = np.random.randint(0, num_classes, num_samples)  # Classes aleatórias
    return X, tf.keras.utils.to_categorical(y, num_classes)

# Criando conjunto de treino e teste
X_train, y_train = generate_synthetic_data(800)
X_test, y_test = generate_synthetic_data(200)

2) Pré-processamento das Imagens

Normalização dos valores dos pixels para o intervalo [0,1].

Aumento de dados para melhorar a robustez do modelo (rotação, reflexão, etc.).

In [ ]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
])

3) Definição do Modelo

Utiliza-se uma CNN baseada em um modelo pré-treinado como ResNet ou MobileNet.

Camadas adicionais são adicionadas para adaptação ao novo conjunto de dados sintético.

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights=None)
base_model.trainable = False  # Congela a base

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(2, activation='softmax')
])

4) Treinamento

Inicialmente, apenas as camadas adicionadas são treinadas (base congelada).

Após algumas épocas, algumas camadas da base são descongeladas para fine-tuning.

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

# Descongelando parte do modelo para fine-tuning
tuning_layers = len(base_model.layers) // 2
for layer in base_model.layers[-tuning_layers:]:
    layer.trainable = True

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Epoch 1/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 68s 2s/step - accuracy: 0.5010 - loss: 0.6934 - val_accuracy: 0.5350 - val_loss: 0.6928
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 71s 2s/step - accuracy: 0.5408 - loss: 0.6927 - val_accuracy: 0.5350 - val_loss: 0.6924
Epoch 3/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - accuracy: 0.5347 - loss: 0.6924 - val_accuracy: 0.5350 - val_loss: 0.6922
Epoch 4/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 85s 2s/step - accuracy: 0.5475 - loss: 0.6917 - val_accuracy: 0.5350 - val_loss: 0.6919
Epoch 5/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - accuracy: 0.5248 - loss: 0.6923 - val_accuracy: 0.5350 - val_loss: 0.6917
Epoch 1/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 120s 4s/step - accuracy: 0.5166 - loss: 0.6972 - val_accuracy: 0.5350 - val_loss: 0.6916
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 82s 3s/step - accuracy: 0.5391 - loss: 0.6872 - val_accuracy: 0.5350 - val_loss: 0.6915
Epoch 3/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 154s 4s/step - accuracy: 0.5798 - loss: 0.6771 - val_accuracy: 0.5350 - val_loss: 0.691

5) Busca por Hiperparâmetros

Variação da taxa de aprendizado, função de ativação e número de camadas adicionadas.

Utilização do TensorBoard para monitoramento dos resultados.

Uso de ferramentas como Keras Tuner para otimização automatizada.

In [ ]:
import keras_tuner as kt
from tensorflow import keras
from tensorflow.keras import layers

def model_builder(hp):
    hp_units = hp.Int('units', min_value=64, max_value=256, step=64)
    model = keras.Sequential([
        base_model,  # Certifique-se de que base_model está definido anteriormente
        layers.GlobalAveragePooling2D(),
        layers.Dense(hp_units, activation='relu'),
        layers.Dense(2, activation='softmax')
    ])
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', [1e-4, 1e-5])),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Criação do tuner
tuner = kt.RandomSearch(model_builder, objective='val_accuracy', max_trials=5, directory='tuner_dir', project_name='stone_scan')
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 10m 35s]
val_accuracy: 0.4650000035762787

Best val_accuracy So Far: 0.5350000262260437
Total elapsed time: 00h 56m 41s


6) Salvamento e Avaliação

O modelo treinado é salvo para uso futuro.

Avaliação em um conjunto de dados sintético separado.

In [ ]:
model.save('stone_scan_model.h5')
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc:.4f}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.4860 - loss: 0.7085
Test Accuracy: 0.5350
